# Import Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Read data

In [4]:
df=pd.read_csv('/kaggle/input/run-or-walk/dataset.csv')

# Data Inspection

In [5]:
df.head()

In [6]:
df.shape

In [7]:
df.info()

Dropping date time and username column

In [8]:
df = df.iloc[:,3:]

In [9]:
df

In [10]:
df.describe().T

# Checking Null Values

In [11]:
df.isna().sum()

No Null values 

# Checking Outliers

In [12]:
plt.figure(figsize=(18,18))
i=1
for x in df.columns:
    plt.subplot(4,4,i)
    sns.boxplot(y=df[x])
    plt.title(x)
    i+=1

No Outliers, all data points looks in clusters

# Visualization

In [13]:
dfv = df.copy()

In [14]:
dfv.activity=dfv.activity.map({0:'Walking',1:'Running'})
dfv.wrist=dfv.wrist.map({0:'Left',1:'Right'})

In [15]:
sns.countplot(dfv.activity)

In [16]:
sns.countplot(y=dfv.wrist,hue=dfv.activity)

Dropping wrist column as it is not that important

In [17]:
df.drop('wrist',axis=1,inplace=True)

In [18]:
df['total_acceleration'] = np.sqrt((df.acceleration_x)**2 + (df.acceleration_y)**2 + (df.acceleration_z)**2)

In [19]:
actdf = df.groupby([dfv.activity])['total_acceleration'].agg({'mean','max'})

In [20]:
actdf

In [21]:
sns.barplot(y=actdf.max(),x=actdf.index)
plt.title('Maximum Acceleration')
plt.show()

In [22]:
sns.barplot(y=actdf.mean(),x=actdf.index)
plt.title('Average Acceleration')
plt.show()

In [23]:
df

# Create X and Y

In [24]:
X = df.iloc[:,1:]

In [25]:
X

In [26]:
Y = df.activity

# Handling Skewness

In [27]:
plt.figure(figsize=(18,18))
i=1
for x in X.columns:
    plt.subplot(4,4,i)
    sns.distplot(X[x])
    plt.title(x)
    i+=1

In [28]:
# Taking Cuberoot Transformation
X = np.cbrt(X)

In [29]:
plt.figure(figsize=(18,18))
i=1
for x in X.columns:
    plt.subplot(4,4,i)
    sns.distplot(X[x])
    plt.title(x)
    i+=1

# Scaling

In [30]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
X = s.fit_transform(X)
X

# Train

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test,Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=10) 

In [32]:
X_train.shape, X_test.shape,Y_train.shape, Y_test.shape

# Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',criterion='entropy')
rf.fit(X_train,Y_train)
Y_pred = rf.predict(X_test)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
rfacc = accuracy_score(Y_test,Y_pred)
print(rfacc)